## Importing Libraries

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import pathlib
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

## Loading Training Data

### Loading urls

In [ ]:
# For Yolo Model.
data_dir = pathlib.Path('/content/drive/MyDrive/Files')
mitosis_train_urls = list(data_dir.glob('train/Mitosis/*.jpg'))
normal_train_urls = list(data_dir.glob('train/Normal/*.jpg'))

# For Res-Net or other model Training.
data_dir2 = pathlib.Path('/content/drive/MyDrive/Files_for_res-net')
data_dir_r2 = pathlib.Path('/content/drive/MyDrive/files_round2_combined')
x_train_urls = list(data_dir2.glob('Training Images/*.jpg'))
x_train_r2_urls_png = list(data_dir_r2.glob('Training Image/*.png'))

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/files_round2_combined/Training.csv')
df['Label']

0      Normal
1     Mitosis
2     Mitosis
3     Mitosis
4     Mitosis
       ...   
61    Mitosis
62     Normal
63     Normal
64     Normal
65    Mitosis
Name: Label, Length: 66, dtype: object

In [ ]:
y = ['Normal','Mitosis','']

In [ ]:
# Importing Y labels from Csv files.
df = pd.read_csv('/content/drive/MyDrive/Files_for_res-net/Train.csv')
y = df['Label']
y_train_names = list(y)
classes = ['Normal','Mitosis']

In [ ]:
# Importing Y labels from Csv files For Round 2.
df_r2 = pd.read_csv('/content/drive/MyDrive/files_round2_combined/Training.csv')
#row = df_r2.loc[df_r2['Image'] == str(x_train_r2_urls_png[0])[-10:],'Label'].iloc[0]

In [ ]:
def names_to_labels(y_train_names):
  for i in range(len(y_train_names)):
    if y_train_names[i] == 'Normal':
      y_train_names[i] = 0
    else:
      y_train_names[i] = 1
  return y_train_names

In [ ]:
# For Round 2
import os
def sorted_labels_generator(df,x_train_urls):
  y_train_names = []
  for image_url in x_train_urls:
    img_url_str = str(image_url)
    filename = os.path.basename(img_url_str) # Get Name
    label = df.loc[df['Image'] == filename , 'Label'].iloc[0]
    y_train_names.append(label)
  y_train_labels = names_to_labels(y_train_names)
  return y_train_labels

In [ ]:
# 0 Label As Normal 1 Label as Mitosis
y_train_labels = names_to_labels(y_train_names)
y_train_labels_r2 = sorted_labels_generator(df_r2,x_train_r2_urls_png)
print(len(y_train_labels))
print(len(y_train_labels_r2))

226
66


### url images to matrix convertion

In [ ]:
def png_to_jpg(training_urls):
  for image_url in training_urls:
    im = Image.open(str(image_url))
    im.save('/content/drive/MyDrive/files_round2_combined/Training Images JPG/' +
            image_url[:-4] + '.jpg',quality=100)
  return None

In [ ]:
def img_to_matrix(training_imgs):
  x_train = []
  for image_url in training_imgs:
    img = cv2.imread(str(image_url))
    img_resized = cv2.resize(img,(420,420))
    x_train.append(img_resized)
  return x_train

In [ ]:
x_mitosis_train = img_to_matrix(mitosis_train_urls)
x_normal_train = img_to_matrix(normal_train_urls)
x_train = img_to_matrix(x_train_urls)  #For Models Apart Yolo.
x_train_r2 = img_to_matrix(x_train_r2_urls_png)

In [ ]:
# Convertion from list to Np.Array
x_mitosis_train = np.array(x_mitosis_train)
x_normal_train = np.array(x_normal_train)
#x_normal_train.shape

In [ ]:
x_mitosis_train.shape

(71, 800, 800, 3)

In [ ]:
x_train = np.array(x_train)
x_train_r2 = np.array(x_train_r2)
x_train.shape
x_train_r2.shape

(66, 420, 420, 3)

## Performing Normalization

In [ ]:
# Spliting For Feeding Data To Esmbling.
from sklearn.model_selection import train_test_split
x_train , x_cross , y_train , y_cross = train_test_split(x_train,y_train_labels,test_size=0.2)
x_train_r2,x_cross_r2,y_train_r2,y_cross_r2 = train_test_split(x_train_r2,y_train_labels_r2,test_size=0.2)

In [ ]:
# Max Scaling
x_train_max = x_train/255
x_cross_max = x_cross/255
x_train_max_r2 = x_train_r2/255
x_cross_max_r2 = x_cross_r2/255

In [ ]:
y_train = np.array(y_train)
y_cross = np.array(y_cross)

In [ ]:
y_train_r2 = np.array(y_train_r2)
y_cross_r2 = np.array(y_cross_r2)

## Data Augmentation

In [ ]:
def data_augment(x_train,save_to_dir,num_augmentations):

  i = 0
  count = 0
  augmented_images = []
  augmented_labels = []

  train_datagen = ImageDataGenerator(
    rotation_range=10,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='reflect',

)

  for batch in train_datagen.flow(x_train, batch_size=1, save_to_dir=save_to_dir, save_prefix='aug', save_format='jpg'):

      batch = batch.reshape(800,800,3)
      augmented_image = batch
      augmented_images.append(augmented_image)
      count = count + 1
      original_image_label = y_train[i]

      if(count ==  num_augmentations):
        count = 0
        i = i + 1

      # Repeat the label for each set of augmented images generated for the original image
      augmented_labels.append(original_image_label)

      if len(augmented_images) == len(x_train) * num_augmentations:
          break

  augmented_images = np.array(augmented_images)
  augmented_labels = np.array(augmented_labels)

  return augmented_images,augmented_labels

In [ ]:
# Augemnetations for YOLO.
save_to_dir_mitosis = '/content/drive/MyDrive/Files_Yolo/train/Mitosis'
save_to_dir_normal =  '/content/drive/MyDrive/Files_Yolo/train/Normal'
num_augmentation = 1

data_augment(x_mitosis_train,save_to_dir_mitosis,num_augmentation)
data_augment(x_normal_train,save_to_dir_normal,num_augmentation)


In [ ]:
# Augmentation for Esembling of models.
num_augmentation = 1
save_to_dir = '/content/drive/MyDrive/Files_for_res-net/aug'  # Directory to save augmented images
aug_imgs,aug_labels = data_augment(x_train,save_to_dir,num_augmentation)
x_train_combined = np.concatenate((x_train,aug_imgs),axis=0)
y_train_combined = np.concatenate((y_train,aug_labels),axis=0)
print(x_train_combined.shape)
print(y_train_combined.shape)

## Esembling Of Models

### Res_Net and Inception Model

In [ ]:
from keras.applications import ResNet50V2,InceptionResNetV2
from keras.models import Model
from keras.layers import MaxPooling2D,GlobalAveragePooling2D,Dense,Dropout,Conv2D,concatenate
from keras.layers import Lambda
# Res-Net Model.
# base_resnet = ResNet50V2(
# #     input_shape=(224,224,3),
# #     include_top=False,
# #     weights='imagenet'
# # )

#Inception Model
base_inception = InceptionResNetV2(
    input_shape=(420,420,3),
    include_top=False,
    weights='imagenet'
)

#Freeze Some Layers of Inception Model
for layer in base_inception.layers:
  layer.trainable = False

# Freeze some Layers of Res-Net Only
# for layer in base_resnet.layers[:5]:
#   layer.trainable = False

# Resize the output of InceptionV3 to match the shape of ResNet50V2

# resized_output_inception = Lambda(lambda x: tf.image.resize(x, (7, 7)))(base_inception.output)

# combined_output = concatenate([base_resnet.output,resized_output_inception])
combined_output = base_inception.output
x = GlobalAveragePooling2D()(combined_output)
x = Dense(256,activation='relu')(x)
x = Dense(units=256,activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1,activation='sigmoid')(x)
model_resnet = Model(inputs=base_inception.input,outputs=predictions)

In [ ]:
model_resnet.compile(
    keras.optimizers.AdamW(0.0001),
    keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
def data_generator(X, y, batch_size):
    num_samples = len(X)
    while True:
        indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_X = X[batch_indices]  # Assuming X is a list of input data for each input of the model
            batch_y = y[batch_indices]
            yield batch_X, batch_y

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7ce71c703a10 to Device at 0x7ce71c7103d0>

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
batch_size = 8
train_generator = data_generator(x_train_max, y_train, batch_size)  # Pass X as a list containing data for each input
model_resnet.fit_generator(train_generator, steps_per_epoch=len(x_train_max) // batch_size, epochs=10)

Epoch 1/10


<ipython-input-48-94c6f9748ef2>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_resnet.fit_generator(train_generator, steps_per_epoch=len(x_train_max) // batch_size, epochs=10)


In [ ]:
batch_size = 8
train_generator = data_generator(x_train_max_r2, y_train_r2, batch_size)  # Pass X as a list containing data for each input
model_resnet.fit_generator(train_generator, steps_per_epoch=len(x_train_max_r2) // batch_size, epochs=10)

In [ ]:
model_resnet.evaluate(x_cross_max_r2,y_cross_r2)

1/1 [==============================] - 6s 6s/step - loss: 2.3318 - accuracy: 0.7143


[2.3318068981170654, 0.7142857313156128]

In [ ]:
y_pred_r1 = model_resnet.predict(x_cross_max)
len(y_pred_r1)

2/2 [==============================] - 4s 46ms/step


46

In [ ]:
y_pred_r1

In [ ]:
y_pred = model_resnet.predict(x_cross_max_r2)
len(y_pred)

1/1 [==============================] - 3s 3s/step


14

In [ ]:
y_cross_r2

array([1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])

### DenseNet201

In [ ]:
from keras.applications import DenseNet201
from keras.models import Model
from keras.layers import MaxPooling2D,GlobalAveragePooling2D,Dense,Dropout,Conv2D

base_denseNet = DenseNet201(
    input_shape=(800,800,3),
    include_top=False,
    weights='imagenet'
)

# Freeze some Layers Only
for layer in base_denseNet.layers[:10]:
  layer.trainable = False

x = base_denseNet.output
x = GlobalAveragePooling2D()(x)
x = Dense(312,activation='relu')(x)
x = Dropout(0.3)(x)
predictions_denseNet = Dense(1,activation='sigmoid')(x)
model_denseNet = Model(input=base_denseNet.inputs,outputs=predictions_denseNet)

In [ ]:
model_denseNet.compile(
    keras.optimizers.AdamW(0.002),
    keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
History_denseNet=model_denseNet.fit(x_train_combined,y_train_combined,epochs=10)

### Inception ResNetV2

In [ ]:
from keras.applications import InceptionResNetV2
from keras.models import Model
from keras.layers import MaxPooling2D,GlobalAveragePooling2D,Dense,Dropout,Conv2D

base_inception = InceptionResNetV2(
    input_shape=(800,800,3),
    include_top=False,
    weights='imagenet'
)

# Freeze some Layers Only
for layer in base_inception.layers[:12]:
  layer.trainable = False

x = base_inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(280,activation='relu')(x)
x = Dropout(0.2)(x)
predictions_inception = Dense(1,activation='sigmoid')(x)
model_inception = Model(input=base_inception.inputs,outputs=predictions_inception)

In [ ]:
model_inception.compile(
    keras.optimizers.AdamW(0.002),
    keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
History_inception=model_inception.fit(x_train_combined,y_train_combined,epochs=10)

## YOLO Model Implementation

### Importing Yolo

In [ ]:
from ultralytics import YOLO

### Training yolo

In [ ]:
data_path = '/content/drive/MyDrive/Files_2'
model = YOLO('/content/runs/classify/train6/weights/best.pt',task='classify')
args_dict = {
    'epochs': 10,
    'imagsz': 128,
    'batch': -1,
    'verbose': True,
    'single_cls': True,
    'close_mosaic': 4,
    'lr0': 0.05,
    'weight_decay': 0.0009,
    'dropout': 0.03,
    'plots': True,
    'amp' : False,
    'cos_lr':True
}
result = model.train(data=data_path, epochs=args_dict['epochs'], imgsz=args_dict['imagsz'],
                     batch=args_dict['batch'], verbose=args_dict['verbose'],
                     single_cls=args_dict['single_cls'], close_mosaic=args_dict['close_mosaic'],
                     lr0=args_dict['lr0'], weight_decay=args_dict['weight_decay'],
                     dropout=args_dict['dropout'], plots=args_dict['plots'],amp=args_dict['amp'],
                     cos_lr=args_dict['cos_lr'])

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=/content/runs/classify/train6/weights/best.pt, data=/content/drive/MyDrive/Files_2, epochs=10, time=None, patience=100, batch=-1, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=4, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.03, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

train: Scanning /content/drive/MyDrive/Files_2/train... 56 images, 0 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Files_2/val... 8 images, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.05' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0006046875), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 128 train, 128 val
Using 2 dataloader workers
Logging results to runs/classify/train8
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]

                   all       0.75          1


...

      10/10      5.83G     0.9646         13        128: 100%|██████████| 2/2 [00:00<00:00,  3.46it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 37.31it/s]

                   all      0.875          1



10 epochs completed in 0.016 hours.
Optimizer stripped from runs/classify/train8/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train8/weights/best.pt, 112.5MB

Validating runs/classify/train8/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56125762 parameters, 0 gradients, 153.8 GFLOPs
train: /content/drive/MyDrive/Files_2/train... found 56 images in 2 classes ✅ 
val: /content/drive/MyDrive/Files_2/val... found 8 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 18.29it/s]


                   all      0.875          1
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train8
Results saved to runs/classify/train8


### Validating Yolo

### Testing yolo

In [ ]:
modelPath = '/content/runs/classify/train2/weights/best.pt'
model = YOLO(modelPath)
result = model.predict('/content/drive/MyDrive/Files_2/Testing Image')
result[0]


image 1/99 /content/drive/MyDrive/Files_2/Testing Image/0_0.png: 128x128 Normal 0.95, Mitosis 0.05, 23.8ms
image 2/99 /content/drive/MyDrive/Files_2/Testing Image/0_1500.png: 128x128 Mitosis 0.77, Normal 0.23, 15.2ms
image 3/99 /content/drive/MyDrive/Files_2/Testing Image/0_2500.png: 128x128 Normal 0.56, Mitosis 0.44, 12.3ms
image 4/99 /content/drive/MyDrive/Files_2/Testing Image/0_3500.png: 128x128 Normal 0.77, Mitosis 0.23, 12.7ms
image 5/99 /content/drive/MyDrive/Files_2/Testing Image/0_4000.png: 128x128 Mitosis 0.81, Normal 0.19, 13.5ms
image 6/99 /content/drive/MyDrive/Files_2/Testing Image/0_5000.png: 128x128 Mitosis 0.93, Normal 0.07, 12.1ms
image 7/99 /content/drive/MyDrive/Files_2/Testing Image/1000_1500.png: 128x128 Normal 0.51, Mitosis 0.49, 16.1ms
image 8/99 /content/drive/MyDrive/Files_2/Testing Image/1000_3000.png: 128x128 Mitosis 0.82, Normal 0.18, 17.2ms
image 9/99 /content/drive/MyDrive/Files_2/Testing Image/1000_4000.png: 128x128 Mitosis 0.87, Normal 0.13, 12.2ms
ima

ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'Mitosis', 1: 'Normal'}
obb: None
orig_img: array([[[139,  81, 152],
        [138,  79, 153],
        [138,  77, 151],
        ...,
        [150,  79, 159],
        [152,  81, 161],
        [154,  85, 166]],

       [[128,  72, 153],
        [127,  71, 152],
        [124,  69, 148],
        ...,
        [164,  99, 175],
        [166, 101, 180],
        [170, 105, 184]],

       [[121,  67, 150],
        [121,  65, 148],
        [116,  60, 141],
        ...,
        [172, 113, 188],
        [173, 115, 193],
        [176, 118, 196]],

       ...,

       [[170, 111, 179],
        [165, 108, 177],
        [159, 110, 178],
        ...,
        [229, 206, 228],
        [232, 206, 230],
        [233, 205, 234]],

       [[170, 111, 179],
        [165, 108, 177],
        [161, 110, 178],
        ...,
        [226, 204, 228],
        [229, 204, 230],
        [229, 203, 233]],

       

In [ ]:
import pickle
# import threading

In [ ]:
with open('model_pickle','wb') as f:
  pickle.dump(model,f)

TypeError: cannot pickle '_thread.lock' object

## Storing Results

In [ ]:
probss = result[0].probs
print(probss.data)
print(result[0].names)

tensor([0.0015, 0.9985], device='cuda:0')
{0: 'Mitosis', 1: 'Normal'}


In [ ]:
y_predictions = []
for i in range(len(result)):
  y_predictions.append(result[i].probs.data.argmax())

In [ ]:
len(y_predictions)

57

In [ ]:
y_prediction_labels = []
for i in range(len(y_predictions)):
  y_prediction_labels.append(y_predictions[i].item())
y_prediction_labels

In [ ]:
#print(y_predictions[0])
df = pd.read_csv('/content/drive/MyDrive/Test.csv')

In [ ]:
for i in range(len(y_prediction_labels)):
  if y_prediction_labels[i] == 1:
    y_prediction_labels[i] = 'Normal'
  else:
    y_prediction_labels[i] = 'Mitosis'

In [ ]:
y_prediction_labels

In [ ]:
df['Label'] = y_prediction_labels

In [ ]:
df

In [ ]:
df.to_csv('yolox8_736_tuned.csv', index=False)